In [2]:
from math import *

def quaternion2rpy(w, x, y, z, form = 'Deg'):
	roll = atan2((-2 * y * z + 2 * x * w), (1 - 2 * x * x - 2 * y * y))
	pitch = asin(-2 * x * z + 2 * w * y)
	yaw = atan2((2 * x * y + 2 * w * z), (-2 * y * y - 2 * z * z + 1))
	if form == 'Deg':
		roll = degrees(roll)
		pitch = degrees(pitch)
		yaw = degrees(yaw)
	if yaw<0: yaw = 360 + yaw
	return {'roll': roll, 'pitch': pitch, 'yaw': yaw}

def rpy2quaternion(roll, pitch, yaw, form = 'Deg'):
	if form == 'Deg':
		roll = radians(roll) / 2
		pitch = radians(pitch) / 2
		yaw = radians(yaw) / 2
	elif form == 'Rad':
		roll = roll / 2
		pitch = pitch / 2
		yaw = yaw / 2
	w = cos(roll) * cos(pitch) * cos(yaw) + sin(roll) * sin(pitch) * sin(yaw)
	x = sin(roll) * cos(pitch) * cos(yaw) - cos(roll) * sin(pitch) * sin(yaw)
	y = cos(roll) * sin(pitch) * cos(yaw) + sin(roll) * cos(pitch) * sin(yaw)
	z = cos(roll) * cos(pitch) * sin(yaw) - sin(roll) * sin(pitch) * cos(yaw)
	return {'w': w, 'y': y, 'z': z, 'x': x}
'''
for i in range(360):
	_degree = i / 2
	sin_degree = sin(radians(_degree))
	cos_degree = cos(radians(_degree))
	dic = rpy2quaternion(0,0,i)
	D_A = quaternion2rpy(cos_degree, 0 ,0 , sin_degree)
	F_A = quaternion2rpy(dic['w'], dic['x'], dic['y'], dic['z'])
	print "Angle: %s Direct : z: %s w: %s  D_A: %s Func: z: %s w: %s F_A: %s" % (i, sin_degree, cos_degree, D_A['yaw'], dic['z'],dic['w'], F_A['yaw'])
'''

'\nfor i in range(360):\n\t_degree = i / 2\n\tsin_degree = sin(radians(_degree))\n\tcos_degree = cos(radians(_degree))\n\tdic = rpy2quaternion(0,0,i)\n\tD_A = quaternion2rpy(cos_degree, 0 ,0 , sin_degree)\n\tF_A = quaternion2rpy(dic[\'w\'], dic[\'x\'], dic[\'y\'], dic[\'z\'])\n\tprint "Angle: %s Direct : z: %s w: %s  D_A: %s Func: z: %s w: %s F_A: %s" % (i, sin_degree, cos_degree, D_A[\'yaw\'], dic[\'z\'],dic[\'w\'], F_A[\'yaw\'])\n'

In [3]:
%matplotlib inline

import os,sys
import os.path as osp

import argparse
import os, tempfile, glob, shutil
import random
from datetime import datetime

import GPy
import GPyOpt
import subprocess as sub
import numpy as np
import scipy.spatial
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy import optimize
import time
import socket
from sklearn.neighbors import NearestNeighbors

global g_blender_executable_path,  blank_file, render_code, model_file, table_file, temp_dirname, epsilon, \
       filename, location_gt, rotation_gt, obj_start_config_array, hand_start_config_array, \
       obj_end_config_array, hand_end_config_array, id_of_sample, train_file, init_pose_file, final_pose_file

epsilon = 1e-6


In [4]:

if os.environ.get('BLENDER_PATH') == None:
    print("Please set BLENDER_PATH in bashrc!")
    sys.exit()
g_blender_executable_path = os.environ['BLENDER_PATH']

if os.environ.get('PHYSICS_PATH') == None:
    print("Please set PHYSICS_PATH in bashrc!")
    sys.exit()

g_repo_path = os.environ['PHYSICS_PATH']

# Input Pose File path
init_pose_file = g_repo_path + '/tmp/init_pose.txt'
final_pose_file = g_repo_path + '/tmp/final_pose.txt'

g_blank_blend_file_path = g_repo_path + '/blank.blend'
obj_model_folder = g_repo_path+'/obj_models/'
table_model_folder = g_repo_path+'/table_models/'
model_name = 'expo_dry_erase_board_eraser'
robot_name = 'reflex'
table_name = 'table'


blank_file = osp.join(g_blank_blend_file_path)
render_code = g_repo_path + '/render_gpyopt_dynamics_reflex_expo_socket.py'
model_file = osp.join(obj_model_folder, model_name+ '.obj')
robot_file = osp.join(obj_model_folder, robot_name+ '.obj')
table_file = osp.join(table_model_folder, table_name+ '.obj')
temp_dirname = '/home/zhusj/repos/blender_scene_generator/test'  #tempfile.mkdtemp()

In [5]:
# Peter KT Yu, Aug 2015
# Plot the trajectory

import numpy as np
import json

import matplotlib
# import matplotlib.patches as mpatches
# import matplotlib.pyplot as plt
# from matplotlib.collections import PatchCollection

# from config.shape_db import ShapeDB

import tf.transformations as tfm
from ik.helper import *
from config.helper import *
# from matplotlib.pyplot import savefig
import time

# from mpl_toolkits.mplot3d import Axes3D

import pdb
def extract2d_and_cleanup(data):

    # print "data: ", data
    tip_pose = data['tip_pose']
    object_pose = data['object_pose']

#     print "obj pose: ", object_pose[0]
    ft = data['ft_wrench']
    
    # transformation to the first 
    invT0 = np.linalg.inv(matrix_from_xyzquat(object_pose[0][1:4], object_pose[0][4:8]))
    sub = 1
    
    # object
    object_pose_2d = []
    for i in (range(0, len(object_pose), sub)):
        T = matrix_from_xyzquat(object_pose[i][1:4], object_pose[i][4:8])
        tip_pose_object0 = np.dot(invT0, T)
        scale, shear, angles, trans, persp = tfm.decompose_matrix(tip_pose_object0)
        #print 'trans', trans[0:2], 'angle', angles[2]
        time = object_pose[i][0]
        # don't add redundant data entry with the same time
        if(not(len(object_pose_2d) > 0 and time == object_pose_2d[-1][0] )):
            object_pose_2d.append([time] + trans[0:2].tolist() + [angles[2]])
    
    # probe
    tip_pose_2d = []
    for i in (range(0, len(tip_pose), sub)):
        tip_pose_0 = np.dot(invT0, tip_pose[i][1:4]+[1])
        #print 'trans', tip_pose_0[0:2]
        time = tip_pose[i][0]
        
        # don't add redundant data entry with the same time
        if(not(len(tip_pose_2d) > 0 and time == tip_pose_2d[-1][0] )):
            tip_pose_2d.append([time] + tip_pose_0[0:2].tolist())

    # ft, no redundency
    ft_2d = np.array(ft)[:,0:3].tolist()   # only need the force
#     print 'object_pose_2d', len(object_pose_2d), 'tip_pose_2d', len(tip_pose_2d), 'ft_2d', len(ft_2d)
    print 'object_pose_2d', len(object_pose_2d[0]), 'tip_pose_2d', len(tip_pose_0), 'ft_2d', len(ft_2d)

    data2d = {}
    data2d['tip_poses_2d'] = tip_pose_2d
    data2d['object_poses_2d'] = object_pose_2d
    data2d['force_2d'] = ft_2d
    return data2d


import pandas as pd

def resample_using_pandas(data):
    force_2d = data['force_2d']
    object_poses_2d = data['object_poses_2d']
    tip_poses_2d = data['tip_poses_2d']
    starttime = max(tip_poses_2d[0][0], object_poses_2d[0][0], force_2d[0][0])
    endtime = min(tip_poses_2d[-1][0], object_poses_2d[-1][0], force_2d[-1][0])
    
    pd_starttime = pd.to_datetime(starttime, unit='s')
    pd_endtime = pd.to_datetime(endtime, unit='s')
    
    tip_poses_2d_dt = pd.to_datetime(np.array(tip_poses_2d)[:,0].tolist(), unit='s')    
    tip_poses_2d = pd.DataFrame(np.array(tip_poses_2d)[:,1:3].tolist(), index=tip_poses_2d_dt)
    tip_poses_2d_resampled = tip_poses_2d.resample('10ms', how='mean')
    tip_poses_2d_interp = tip_poses_2d_resampled.interpolate()
    
    start_ = tip_poses_2d_interp.index.searchsorted(pd_starttime)
    end_ = tip_poses_2d_interp.index.searchsorted(pd_endtime)
    tip_poses_2d_interp = tip_poses_2d_interp.ix[start_:end_]
    tip_poses_2d_interp_list = tip_poses_2d_interp.values.tolist()
    
    object_poses_2d_dt = pd.to_datetime(np.array(object_poses_2d)[:,0].tolist(), unit='s')
    object_poses_2d = pd.DataFrame(np.array(object_poses_2d)[:,1:4].tolist(), index=object_poses_2d_dt)
    object_poses_2d_resampled = object_poses_2d.resample('10ms', how='mean')
    object_poses_2d_interp = object_poses_2d_resampled.interpolate()
    start_ = object_poses_2d_interp.index.searchsorted(pd_starttime)
    end_ = object_poses_2d_interp.index.searchsorted(pd_endtime)
    object_poses_2d_interp = object_poses_2d_interp.ix[start_:end_]
    object_poses_2d_interp_list = object_poses_2d_interp.values.tolist()
    
    force_dt = pd.to_datetime(np.array(force_2d)[:,0].tolist(), unit='s')
    force_2d = pd.DataFrame(np.array(force_2d)[:,1:3].tolist(), index=force_dt)
    force_2d_resampled = force_2d.resample('10ms', how='mean')
    force_2d_interp = force_2d_resampled.interpolate()
    start_ = force_2d_interp.index.searchsorted(pd_starttime)
    end_ = force_2d_interp.index.searchsorted(pd_endtime)
    force_2d_interp = force_2d_interp.ix[start_:end_]
    force_2d_interp_list = force_2d_interp.values.tolist()
    
    
    data_resample = {}
    data_resample['tip_poses_2d'] = tip_poses_2d_interp_list
    data_resample['object_poses_2d'] = object_poses_2d_interp_list
    data_resample['force_2d'] = force_2d_interp_list
    return data_resample
    
def animate_2dsynced(data, shape_id, figfname):
    fig, ax = plt.subplots()
    probe_radius = 0.004745   # probe1: 0.00626/2 probe2: 0.004745
    
    v = int(figfname.split('_')[-4].split('=')[1])
    sub = 1                 # subsample rate
    tip_pose = data['tip_poses_2d']
    object_pose = data['object_poses_2d']
    force = data['force_2d']
    
    patches = []
    
    
    # add the object as polygon
    shape_db = ShapeDB()
    shape_polygon = shape_db.shape_db[shape_id]['shape_poly'] # shape of the objects presented as polygon.
    shape_polygon_3d = np.hstack((np.array(shape_polygon), np.zeros((len(shape_polygon), 1)), np.ones((len(shape_polygon), 1))))
    

    print 'object_pose', len(object_pose), 'tip_pose', len(tip_pose), 'force', len(force)
    plt.ion()
    for i in (range(0, len(tip_pose), sub)):
        
        plt.cla()
        T = tfm.compose_matrix(translate = object_pose[i][0:2] + [0], angles = (0,0,object_pose[i][2]) )
        shape_polygon_3d_world = np.dot(T, shape_polygon_3d.T)
        
        obj = mpatches.Polygon(shape_polygon_3d_world.T[:,0:2], closed=True, color='blue', alpha=0.05)
        ax.add_patch(obj)
    
        # add the probes as circle
        circle = mpatches.Circle(tip_pose[i][0:2], probe_radius, ec="none", color='red', alpha=0.5)
        ax.add_patch(circle)
        
        # add the force
        ax.arrow(tip_pose[i][0], tip_pose[i][1], force[i][0]/100, force[i][1]/100, head_width=0.005, head_length=0.01, fc='k', ec='k')
        
        #arrow = mpatches.Arrow(tip_pose[i][0], tip_pose[i][1], force[i][0],
        #                force[i][1], head_width=0.05, head_length=0.1, fc='k', ec='k')
        #ax.add_patch(arrow)
        
        # render it
        plt.axis([-0.1, 0.1, -0.1, 0.1])
        #plt.axis('equal')
        plt.draw()
        #time.sleep(0.1)
    plt.show()

def wraptopi(data):
    return ( data + np.pi) % (2 * np.pi ) - np.pi 

def extend_with_velocity(data, h):
    h = 0.01  # from 10ms
    
    tip_pos = np.array(data['tip_poses_2d'])
    object_pos = np.array(data['object_poses_2d'])
    #nd = tip_pose.shape[0]
    
    tip_vel = (tip_pos[2:,:] - tip_pos[0:-2,:]) / 2 / h
    tip_pos = tip_pos[1:-1,:]
    
    object_vel_xy = (object_pos[2:,0:2] - object_pos[0,0:2]) / 2 / h
    object_vel_th = wraptopi(object_pos[2:,2:] - object_pos[0,2:]) / 2 / h
    object_pos = object_pos[1:-1,:]
    
    data_extended = {}
    data_extended['tip_poses_2d'] = tip_pos.tolist()
    data_extended['tip_vels_2d_vel'] = tip_vel.tolist()
    data_extended['object_poses_2d'] = object_pos.tolist()
    data_extended['object_vels_2d'] = np.hstack((object_vel_xy, object_vel_th)).tolist()
    data_extended['force_2d'] = data['force_2d'][1:-1]
    
    return data_extended


# data: synced 2d data
def extract_training_data(data):    
    tip_pose = data['tip_poses_2d']
    tip_vel = data['tip_vels_2d_vel']
    object_pose = data['object_poses_2d']
    object_vel = data['object_vels_2d']
    force = data['force_2d']
    
    
    # find the beginning of contact by force magnitude
    threshold = 0.5
    start_index = 0
    end_index = 0
    sub = 5
    for i in (range(0, len(force)-1, 1)):
        if norm(np.array(tip_pose[i]) - np.array(tip_pose[i+1]))>0 and norm(np.array(object_pose[i]) - np.array(object_pose[i+1]))>1e-3 and norm(force[i]) > threshold:
            start_index = i
            break
            
    for i in (range(len(force)-2, -1, -1)):
        if norm(np.array(tip_pose[i]) - np.array(tip_pose[i+1]))>0 and norm(np.array(object_pose[i]) - np.array(object_pose[i+1]))>1e-3 and norm(force[i]) > threshold:
            end_index = i
            break
#     print 'start_index', start_index, 'end_index', end_index, 'len', len(force)
    
    data_training = []  
    labels = ['$x$', '$y$', '$\Delta x$', '$\Delta y$', 'force $x$', 'force $y$', r'$\Delta x$', r'$\Delta y$', r'$\Delta \theta$',
        '$v_x$', '$v_y$', '$v_x$', '$v_y$',  # 9:  tip_svx, tip_svy, tip_evx, tip_evy, 
        '$v_x$', '$v_y$', r'$\omega$',       # 13: object_pose_svx, object_pose_svy, object_pose_svtheta, # start speed
        '$v_x$', '$v_y$', r'$\omega$']       # 16: object_pose_evx, object_pose_evy, object_pose_evtheta  # end speed]
                                             # 19: tip_speedset (mm)
    
    for i in (range(start_index, end_index+1-sub, sub)):
        tip_pose_i_obji = transform_to_frame2d(tip_pose[i], object_pose[i])
        tip_pose_isub_obji = transform_to_frame2d(tip_pose[i+sub], object_pose[i])
        
        tip_vel_i_obji = rotate_to_frame2d(tip_vel[i], object_pose[i])
        tip_vel_isub_obji = rotate_to_frame2d(tip_vel[i+sub], object_pose[i])
        
        force_i_obji = rotate_to_frame2d(force[i], object_pose[i])
        
        object_pose_i_obji = transform_to_frame2d(object_pose[i][0:2], object_pose[i])
        object_pose_isub_obji = transform_to_frame2d(object_pose[i+sub][0:2], object_pose[i])
        
        object_vel_i_obji = rotate_to_frame2d(object_vel[i][0:2], object_vel[i])
        object_vel_isub_obji = rotate_to_frame2d(object_vel[i+sub][0:2], object_vel[i])
        
        newdata = tip_pose_i_obji + (np.array(tip_pose_isub_obji) - np.array(tip_pose_i_obji)).tolist() +\
        force_i_obji + (np.array(object_pose_isub_obji) - np.array(object_pose_i_obji)).tolist() + \
        [object_pose[i+sub][2] - object_pose[i][2]] + \
        tip_vel_i_obji + tip_vel_isub_obji + \
        object_vel_i_obji + [object_vel[i][2]] + object_vel_isub_obji + [object_vel[i+sub][2]]
        
        #print newdata
        data_training.append(newdata)
        
    return data_training

def extend_with_speedset(data, v):
    return [d + [v] for d in data]

def json2trainingdata(filepath):
        
    with open(filepath) as data_file:    
        data = json.load(data_file)
    
    shape_id = 'rect1'
    v = getfield_from_filename(filepath, 'v')
    data2d = extract2d_and_cleanup(data)
    data_synced = resample_using_pandas(data2d)
    #animate_2dsynced(data_synced, shape_id, filepath.replace('.json', '.png'))
    data_synced = extend_with_velocity(data_synced, h=0.01)
    
    data_training = extract_training_data(data_synced)
    data_training = extend_with_speedset(data_training, int(v))
    return data_synced
#     return data_training




    


In [6]:
import glob

path = '/home/zhusj/data/push/abs/rect1/rect1_json'

filelist = glob.glob("%s/motion*.json" % path)

all_training_data = []
no_of_samples = 100



for json_filepath in filelist:
    # only process v=20
    # if json_filepath.find('v=20_') == -1:
        # continue

    # try:
    if len(all_training_data) < no_of_samples:
#     print 'json_filepath', json_filepath

        data_synced = json2trainingdata(json_filepath)
        all_training_data.append(data_synced)
#         all_training_data.extend(json2trainingdata(json_filepath))
#         print 'len(all_training_data)', len(all_training_data)
    else:
        break
    # except:
        # print json_filepath
        # pass
#     print len(data_synced['object_poses_2d'])
#     print 'data_synced', data_synced
#     print 'data_training', data_training

    
#     X_obj = data_synced['object_poses_2d'][0]
#     Y_obj = data_synced['object_poses_2d'][-1]
    
#     X_tip = data_synced['tip_poses_2d'][0]
#     Y_tip = data_synced['tip_poses_2d'][-1]
    
#     t = len(data_synced['object_poses_2d'])*0.01
    
    

#     print X_obj, Y_obj, X_tip, Y_tip, t
#     raw_input("Press Enter to continue...")


# outputfile= "%s/data_training_with_vel.json" % argv[1]
# with open(outputfile, 'w') as outfile:
#     json.dump(all_training_data, outfile, indent=4)
no_of_training_samples = 20

samples = np.random.choice(len(all_training_data), no_of_samples, replace=False)
train_id = samples[:no_of_training_samples]
test_id = samples[no_of_training_samples:]

train_samples = []
test_samples = []
for i,val in enumerate(train_id):
    train_samples.append(all_training_data[val])
for i,val in enumerate(test_id):
    test_samples.append(all_training_data[val])

object_pose_2d 4 tip_pose_2d 4 ft_2d 165
object_pose_2d 4 tip_pose_2d 4 ft_2d 175
object_pose_2d 4 tip_pose_2d 4 ft_2d 386
object_pose_2d 4 tip_pose_2d 4 ft_2d 253
object_pose_2d 4 tip_pose_2d 4 ft_2d 218
object_pose_2d 4 tip_pose_2d 4 ft_2d 201
object_pose_2d 4 tip_pose_2d 4 ft_2d 202
object_pose_2d 4 tip_pose_2d 4 ft_2d 1377
object_pose_2d 4 tip_pose_2d 4 ft_2d 176
object_pose_2d 4 tip_pose_2d 4 ft_2d 306
object_pose_2d 4 tip_pose_2d 4 ft_2d 296
object_pose_2d 4 tip_pose_2d 4 ft_2d 292
object_pose_2d 4 tip_pose_2d 4 ft_2d 172
object_pose_2d 4 tip_pose_2d 4 ft_2d 250
object_pose_2d 4 tip_pose_2d 4 ft_2d 149
object_pose_2d 4 tip_pose_2d 4 ft_2d 263
object_pose_2d 4 tip_pose_2d 4 ft_2d 733
object_pose_2d 4 tip_pose_2d 4 ft_2d 183
object_pose_2d 4 tip_pose_2d 4 ft_2d 212
object_pose_2d 4 tip_pose_2d 4 ft_2d 292
object_pose_2d 4 tip_pose_2d 4 ft_2d 231
object_pose_2d 4 tip_pose_2d 4 ft_2d 189
object_pose_2d 4 tip_pose_2d 4 ft_2d 242
object_pose_2d 4 tip_pose_2d 4 ft_2d 193
object_pose_2d 

In [7]:
train_samples[0]

{'force_2d': [[-0.04074706999999855, 0.019557890000000133],
  [-0.043816569999998535, -0.013587276666666518],
  [-0.03778906999999854, -0.023565109999999834],
  [-0.043901236666665344, 0.0004610566666667741],
  [-0.031634569999998474, -0.0011746099999999093],
  [-0.03341856999999863, 0.0042847233333334804],
  [-0.04268256999999864, -0.005609109999999917],
  [-0.038987236666665336, -0.024483609999999878],
  [-0.047405069999998606, -0.006244109999999858],
  [-0.04355156999999865, -0.006426609999999888],
  [-0.04606456999999864, -0.011178109999999852],
  [-0.04444090333333195, -0.025605609999999873],
  [-0.017236069999998715, -0.07865060999999984],
  [0.5166947633333348, -0.8397632766666664],
  [1.1119029300000012, -1.2146051099999997],
  [3.5387794300000013, -7.412171943333333],
  [3.643889930000001, -7.69657911],
  [2.6847327633333347, -5.4763769433333325],
  [1.2602344300000015, -2.80769361],
  [-0.6216805699999987, 0.15301505666666682],
  [-0.09520656999999866, 1.45876739],
  [-0.3127

In [104]:
def test( mass, friction):
    print "testing.."
    y_test = []
    y_new = 0
    p_error = 0 
    r_error = 0
    angle_error = 0
    for count in xrange(0, len(test_samples)): 

        data = test_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
#         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
#         raw_input("Press Enter to continue...")
   
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (mass, friction, X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                        x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50007)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(count))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10000)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, 7))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,7):
                    Y_new[0,i] = vals[i]
#         print Y_new
#         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
        location = Y_new[0,0:2]
        location_gt = np.array(Y_obj[0:2])
        rotation = Y_new[0,3:]
        rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])
        
        angles = quaternion2rpy(Y_new[0,3], Y_new[0,4], Y_new[0,5], Y_new[0,6], form = 'Deg')
        angles_gt = quaternion2rpy(y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z'], form = 'Deg')

#         print 'angles: ', angles, angles_gt
            
        dist = np.sqrt(np.sum(((location-location_gt)**2)))
#         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        

        if angles['yaw'] > 180:
            angles['yaw'] -= 360
        if angles_gt['yaw'] > 180:
            angles_gt['yaw'] -= 360 
#         print 'angles: ', angles, angles_gt

        angle_dist = np.abs(angles['yaw'] - angles_gt['yaw'])

#         print dist, q_dist
#         y_new += dist + q_dist


#         print dist, q_dist
        y_test.append(dist + q_dist)
#         print y_test
        y_new += y_test[count]
        count += 1
        p_error += dist
        r_error += q_dist
        angle_error += angle_dist

#         print "y_new", y_new

    return y_new, y_test, p_error, r_error, angle_error

In [103]:
y_new, y_test, p_error, r_error, angle_error = test(0.837, 0.2)

print "current loss:", y_new/(no_of_samples - no_of_training_samples), \
                        p_error/(no_of_samples - no_of_training_samples), \
                        r_error/(no_of_samples - no_of_training_samples), \
                        angle_error/(no_of_samples - no_of_training_samples)

testing..
angles:  {'yaw': 10.771550733576174, 'roll': 0.10415702273663649, 'pitch': 0.10415677151887369} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': 6.0379864275907655, 'roll': 0.0, 'pitch': 0.0} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': 7.088229235894922, 'roll': -0.00021457750712440242, 'pitch': -0.00021457750712247395} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': -0.0245225942657612, 'roll': -0.00011461608162010182, 'pitch': -0.00011461608161987267} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': -0.15790721021500076, 'roll': -0.00011474935160324924, 'pitch': -0.00011474935160302036} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': 2.036639522771315, 'roll': 0.03466138409494893, 'pitch': 0.03466137728500521} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': 4.210468082968538, 'roll': 0.0, 'pitch': 0.0} {'yaw': 0.0, 'roll': 0.0, 'pitch': 0.0}
angles:  {'yaw': -23.07684628601146, 'roll': -0.0005407878192346053, 'pitch': 

In [9]:
def transform_data(data):

    X_obj = data['object_poses_2d'][0]
    Y_obj = data['object_poses_2d'][-1]
    
    x_quat = rpy2quaternion(0, 0, X_obj[2], form = 'Rad')
    y_quat = rpy2quaternion(0, 0, Y_obj[2], form = 'Rad')

    X_tip = data['tip_poses_2d'][0]
    Y_tip = data['tip_poses_2d'][-1]

    t = len(data_synced['object_poses_2d'])*0.01
        
    return X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t    

In [29]:
train_sample_bak = train_samples

In [34]:
len(train_sample_bak)

20

In [86]:
train_samples = train_sample_bak

X_train = np.zeros((len(train_samples), 11))
Y_train = np.zeros((len(train_samples), 6))
X_test = np.zeros((len(test_samples), 11))
Y_test = np.zeros((len(test_samples), 6))

for count in xrange(0, len(train_samples)): 

    data = train_samples[count]

    X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
    
#     print X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t
    
    X_train[count,:] = np.array([X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], x_quat['y'], x_quat['z'], \
                                X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t])
    Y_train[count,:] = np.array([Y_obj[0], Y_obj[1], y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])
    
for count in xrange(0, len(test_samples)): 

    data = test_samples[count]

    X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
    
#     print X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t
    
    X_test[count,:] = np.array([X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], x_quat['y'], x_quat['z'], \
                                X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t])
    Y_test[count,:] = np.array([Y_obj[0], Y_obj[1], y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])

    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X_train)

    distances, indices = nbrs.kneighbors(X_test)
#     print indices
    Y_predict = np.zeros(Y_test.shape) 
    for i in xrange(0,X_test.shape[0]):
        Y_predict[i,:] = Y_train[indices[i,0],:]
#     print Y_predict
#     Y_predict = model.predict(X_test)[0]
    gppred_pos_diff = np.abs(Y_predict[:,0:2] - Y_test[:,0:2])
    ros_diff =  np.multiply(Y_test[:,2:], Y_predict[:,2:])**2
    gppred_rot_diff = 1 - np.sum(ros_diff,axis=1)
    num_of_samples = X_test.shape[0]
    
print np.sum(np.sqrt(np.sum(gppred_pos_diff**2,axis=1)))/len(test_samples), \
       np.sum(gppred_rot_diff)/len(test_samples)

0.0191808155438 0.0


In [87]:
len(train_samples)

20

In [ ]:
def test_NN(train_samples, test_samples):
    
    for count in xrange(0, len(train_samples)): 

        data = train_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
    
    
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X_train[:,0:2])

    obj_start_config_test, hand_start_config_test, obj_end_config_test, hand_end_config_test = \
                                                                                read_data(test_filename)
    X_test = np.squeeze(np.array(obj_start_config_test))
    Y_test = np.squeeze(np.array(obj_end_config_test))
    distances, indices = nbrs.kneighbors(X_test[:,0:2])
#     print indices
    Y_predict = np.zeros(X_test.shape) 
    for i in xrange(0,X_test.shape[0]):
        Y_predict[i,:] = Y_train[indices[i,0],:]
#     print Y_predict
#     Y_predict = model.predict(X_test)[0]
    gppred_pos_diff = np.abs(Y_predict[:,0:2] - Y_test[:,0:2])
    ros_diff =  np.multiply(Y_test[:,3:], Y_predict[:,3:])**2
    gppred_rot_diff = 1 - np.sum(ros_diff,axis=1)
    num_of_samples = X_test.shape[0]
    
    return np.sum(np.sqrt(np.sum(gppred_pos_diff**2,axis=1)))/num_of_samples, \
           np.sum(gppred_rot_diff)/num_of_samples

In [46]:
def myf(X):

    mass = X[0][0]
    friction = X[0][1]
    
    y_new = 0
#     count = 0
#     if new_iter is True:
# #   randomly select (number_of_samples) samples
#         samples = random.sample(xrange(0, len(train_samples)), number_of_samples)
#         new_iter = False 
    
#     for count in samples:  
    for count in xrange(0, number_of_samples):
#         print "input: ", train_file, count, mass, friction, force_gt,count

        data = train_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
#         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
#         raw_input("Press Enter to continue...")
   
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                        x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50007)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(count))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10000)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, 7))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,7):
                    Y_new[0,i] = vals[i]
#         print Y_new
#         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
        location = Y_new[0,0:2]
        location_gt = np.array(Y_obj[0:2])
        rotation = Y_new[0,3:]
        rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])

        
#         print 'location_gt: ', location_gt, rotation_gt
            
        dist = np.sqrt(np.sum(((location-location_gt)**2)))
#         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        


#         print dist, q_dist
        y_new += dist + q_dist
        count += 1

#     print "X, y_new",X,y_new
#     raw_input("Press Enter to continue...")

    return y_new


In [58]:
def myf_GS(X):

    mass = X[0]
    friction = X[1]
    
    y_new = 0
#     count = 0
#     if new_iter is True:
# #   randomly select (number_of_samples) samples
#         samples = random.sample(xrange(0, len(train_samples)), number_of_samples)
#         new_iter = False 
    
#     for count in samples:  
    for count in xrange(0, number_of_samples):
#         print "input: ", train_file, count, mass, friction, force_gt,count

        data = train_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
#         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
#         raw_input("Press Enter to continue...")
   
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (X[0], X[1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                        x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50007)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(count))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10000)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, 7))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,7):
                    Y_new[0,i] = vals[i]
#         print Y_new
#         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
        location = Y_new[0,0:2]
        location_gt = np.array(Y_obj[0:2])
        rotation = Y_new[0,3:]
        rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])

        
#         print 'location_gt: ', location_gt, rotation_gt
            
        dist = np.sqrt(np.sum(((location-location_gt)**2)))
#         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        


#         print dist, q_dist
        y_new += dist + q_dist
        count += 1

#     print "X, y_new",X,y_new
#     raw_input("Press Enter to continue...")

    return y_new


In [105]:
import time

# global new_iter

# new_iter = True

BO_result = []
BO_time =[]
BO_test_loss =[]
from GPyOpt.acquisitions.EI import AcquisitionEI
from GPyOpt.acquisitions.MPI import AcquisitionMPI
force_gt = float(48)


#     import pdb; pdb.set_trace()

############################# GPyOpt setup ######################

from GPyOpt.acquisitions.EI import AcquisitionEI
from GPyOpt.acquisitions.MPI import AcquisitionMPI

space = GPyOpt.Design_space(space =[{'name': 'mass', 'type': 'continuous', 'domain': (0.1,2.0)},
                                    {'name': 'friction', 'type': 'continuous', 'domain': (0.0,1.0)}])

model = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False, normalize_Y = False)

aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space)

from numpy.random import seed # fixed seed
seed(123456)
initial_design = GPyOpt.util.stats.initial_design('random', space, 5)

acquisition = AcquisitionMPI(model, space, optimizer=aquisition_optimizer) 

evaluator = GPyOpt.core.evaluators.Sequential(acquisition)

# objective = GPyOpt.core.task.SingleObjective(myf)
# number_of_samples = 1
# myBopt = GPyOpt.methods.ModularBayesianOptimization(model, space, objective, acquisition, evaluator, \
#                                                     initial_design)

unit_time = 30
unit_iter = 30
max_time = 0
max_iter = 0
eps = 1e-4

number_of_samples = len(train_samples)
# for number_of_samples in xrange(1,len(train_samples)):
for count in xrange(0,10):

    new_iter = True
    print "Using", number_of_samples, "training sample(s)"
#     random.seed(datetime.now())

#     myBopt.run_optimization(max_time=max_time)
    max_time += unit_time
    max_iter += unit_iter
#     eps /= 2
    num_cores = 10
    bounds = [(0.1,2.0),(0.0,1.0)]

    
    objective = GPyOpt.core.task.SingleObjective(myf)
    myBopt = GPyOpt.methods.BayesianOptimization(f=myf,  
                                            bounds = bounds,                  
                                            acquisition_type = 'EI',              
                                            normalize_Y = False,
#                                             initial_design_numdata = 5,
                                            num_cores = num_cores) 

    start_time = time.time()
#     myBopt.run_optimization(max_time=max_time)
#     myBopt = GPyOpt.methods.ModularBayesianOptimization(model, space, objective, acquisition, evaluator, \
#                                                     initial_design)
    myBopt.run_optimization(max_iter=max_iter,max_time=max_time, eps=eps)
    elapsed_time = time.time() - start_time

    print "optima!!!!!!!!!!!!!!!!: ", myBopt.x_opt, myBopt.fx_opt/number_of_samples, " in ", elapsed_time, myBopt.cum_time
    
    y_new, y_test, p_error, r_error, angle_error = test(myBopt.x_opt[0], myBopt.x_opt[1])
    
    print "current loss:", p_error/80, angle_error/80,
    
    BO_test_loss.append([y_new, y_test, p_error, r_error, angle_error])
    BO_result.append(myBopt)
    BO_time.append([myBopt.cum_time,elapsed_time])
    
# exp1: 0.543561 -0.123172 0.54746 0.62389 -0.00114687 0.00659373 -0.781483
# exp2: 0.532153 -0.0257914 0.548445 -0.430238 -0.00623893 -0.00737948 0.902664
# exp3: 0.48143 -0.0221645 0.54965 -0.234441 -0.00235522 -0.00839558 0.972091 

# exp8: 0.423403 0.229149 0.558357 -0.232128 -0.00738495 -0.0275467 0.972267 

Using 20 training sample(s)
optima!!!!!!!!!!!!!!!!:  [ 0.1  1. ] [ 0.02081588]  in  8.74024391174 8.33434796333
testing..
current loss: 0.0126942957023 6.22006103464 Using 20 training sample(s)
optima!!!!!!!!!!!!!!!!:  [ 1.60535432  1.        ] [ 0.01857149]  in  64.4174740314 64.417263031
testing..
current loss: 0.0136283291321 5.74112281301 Using 20 training sample(s)
optima!!!!!!!!!!!!!!!!:  [ 0.83717254  0.86007636] [ 0.02016292]  in  8.622620821 8.37713885307
testing..
current loss: 0.0125935155167 6.4839585102 Using 20 training sample(s)
optima!!!!!!!!!!!!!!!!:  [ 1.36304339  0.99942582] [ 0.01771514]  in  123.104247093 123.10395503
testing..
current loss: 0.0134706721658 5.92204788369 Using 20 training sample(s)
optima!!!!!!!!!!!!!!!!:  [ 0.10587343  0.97593975] [ 0.02014646]  in  13.222522974 12.8564760685
testing..
current loss: 0.0126524304676 6.05601435653 Using 20 training sample(s)
optima!!!!!!!!!!!!!!!!:  [ 1.5150748   0.84018866] [ 0.02001206]  in  8.11692690849 7.806995

In [85]:
len(train_samples)

11

In [54]:
for count in xrange(1,5):
    random.seed(datetime.now())
    start_time = time.time()
    best_y = 10000
    best_f = 2.0
    best_m = 1.0
    result = []
    elapsed_time = 0
    X = np.zeros((1,2))
    
    ################## Grid Search #############
    while elapsed_time < 10:
        m = 0.5*random.random()
        f = 0.7*random.random()
        X[0][0] = m
        X[0][1] = f

        y = myf(X)
        if y < best_y:
            best_y = y
            best_m = m
            best_f = f
            
        result.append([m,f,y,best_m,best_f,best_y])
        elapsed_time = time.time() - start_time
    print "optima!!!!!!!!!!!!!!!!: ", best_m,best_f,best_y
    RS_result.append(result)

optima!!!!!!!!!!!!!!!!:  0.145976401682 0.513319939877 0.111046947666


NameError: name 'RS_result' is not defined

In [69]:
BO_result = []
GS_result =[]
BO_time =[]
GS_time =[]
mass_GT =[]
friction_GT =[]

for number_of_samples in xrange(1,len(train_samples)):
    new_iter = True
    print "Using", number_of_samples, "training sample(s)"
#     for count in xrange(1,11):
    random.seed(datetime.now())
    ################## Grid Search #############
    rranges = (slice(0.8, 0.9, 0.01), slice(0.1, 0.2, 0.01))


    start_time = time.time()

    x0,fval,grid,jout = optimize.brute(myf_GS, rranges, full_output=True, finish=None)

    elapsed_time = time.time() - start_time
    print elapsed_time
    print x0,fval/number_of_samples
    GS_result.append([x0,fval,grid,jout])
    GS_time.append(elapsed_time)

Using 1 training sample(s)
19.5032520294
[ 0.88  0.11] 0.0175345421382
Using 2 training sample(s)
34.783190012
[ 0.85  0.17] 0.0262979035867
Using 3 training sample(s)
46.3919789791
[ 0.85  0.17] 0.0272136327392
Using 4 training sample(s)
54.6441221237
[ 0.85  0.15] 0.0353898771177
Using 5 training sample(s)
79.5296559334
[ 0.82  0.15] 0.0449771234266
Using 6 training sample(s)
91.3373091221
[ 0.82  0.15] 0.0491708137548
Using 7 training sample(s)
108.119342089
[ 0.82  0.15] 0.0590058354344
Using 8 training sample(s)
126.778500795
[ 0.86  0.15] 0.0680493560935
Using 9 training sample(s)
143.340984821
[ 0.81  0.15] 0.0866613854128
Using 10 training sample(s)
144.135673046
[ 0.86  0.15] 0.0870296416253


In [31]:
number_of_samples

1

In [ ]:
       print 'input: ', X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                         x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
